# Exploring quantum chemistry simulations

2024-06-04

In [1]:
# ! pip install --prefer-binary pyscf
# ! pip install geometric
# ! pip install pyberny

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 18.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 40.9 MB/s eta 0:00:0000:01:00:01


In [1]:
import pyscf
from pyscf import gto, scf, dft

## Exploration, methane molecule

In [6]:
molecule = gto.M(atom='O 0 0 0; C 0 0 1; C 0 0 -1', basis='6-311G')

In [17]:
molecule

In [27]:
scf.RHF?

Signature: scf.RHF(mol, *args)
Docstring:
SCF base class.   non-relativistic RHF.

Attributes:
    verbose : int
        Print level.  Default value equals to :class:`Mole.verbose`
    max_memory : float or int
        Allowed memory in MB.  Default equals to :class:`Mole.max_memory`
    chkfile : str
        checkpoint file to save MOs, orbital energies etc.  Writing to
        chkfile can be disabled if this attribute is set to None or False.
    conv_tol : float
        converge threshold.  Default is 1e-9
    conv_tol_grad : float
        gradients converge threshold.  Default is sqrt(conv_tol)
    max_cycle : int
        max number of iterations.  If max_cycle <= 0, SCF iteration will
        be skipped and the kernel function will compute only the total
        energy based on the initial guess. Default value is 50.
    init_guess : str
        initial guess method.  It can be one of 'minao', 'atom', 'huckel', 'hcore', '1e', 'chkfile'.
        Default is 'minao'
    DIIS : DIIS c

In [29]:
rhf = scf.RHF(molecule)
e = rhf.kernel()

converged SCF energy = -149.98193184835


In [26]:
e

-149.98193184834994

### DFT

In [21]:
rks = dft.RKS(molecule)
rks.xc = 'b3lyp'

In [30]:
rks = rks.kernel()

converged SCF energy = -150.810569957495


### Optimise: test code from github

In [39]:
from pyscf import gto, scf
from pyscf.geomopt.berny_solver import optimize

In [40]:
%%time

mol = gto.M(atom='N 0 0 0; N 0 0 1.2', basis='ccpvdz')
mf = scf.RHF(mol)

#
# geometry optimization for HF.  There are two entries to invoke the berny
# geometry optimization.
#
# method 1: import the optimize function from pyscf.geomopt.berny_solver
mol_eq = optimize(mf)
print(mol_eq.atom_coords())


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   N   0.000000   0.000000   1.200000    0.000000  0.000000  0.000000
converged SCF energy = -108.914051975052
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 N     0.0000000000    -0.0000000000    -0.3121321606
1 N    -0.0000000000     0.0000000000     0.3121321606
----------------------------------------------
cycle 1: E = -108.914051975  dE = -108.914  norm(grad) = 0.441422

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N   0.000000   0.000000   0.079377    0.000000  0.000000  0.079377
   N   0.000000   0.000000   1.120623    0.000000  0.000000 -0.079377
converged SCF energy = -108.95047828901
--------------- RHF_Scanner gradients ---------------
       

In [41]:
from pyscf import mcscf

In [42]:
%%time

# method 2: create the optimizer from Gradients class
mol_eq = mf.Gradients().optimizer(solver='berny').kernel()

#
# geometry optimization for CASSCF
#

mf = scf.RHF(mol)
mc = mcscf.CASSCF(mf, 4, 4)

conv_params = {
    'gradientmax': 6e-3,  # Eh/Bohr
    'gradientrms': 2e-3,  # Eh/Bohr
    'stepmax': 2e-2,      # Bohr
    'steprms': 1.5e-2,    # Bohr
}
# method 1
mol_eq = optimize(mc, **conv_params)

# method 2
mol_eq = mc.Gradients().optimizer(solver='berny').kernel(conv_params)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   N   0.000000   0.000000   1.200000    0.000000  0.000000  0.000000
converged SCF energy = -108.914051975052
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 N    -0.0000000000    -0.0000000000    -0.3121321606
1 N     0.0000000000     0.0000000000     0.3121321606
----------------------------------------------
cycle 1: E = -108.914051975  dE = -108.914  norm(grad) = 0.441422

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N   0.000000   0.000000   0.079377    0.000000  0.000000  0.079377
   N   0.000000   0.000000   1.120623    0.000000  0.000000 -0.079377
converged SCF energy = -108.95047828901
--------------- RHF_Scanner gradients ---------------
       

### Optimise own

In [45]:
from pyscf.geomopt.geometric_solver import optimize

In [50]:
molecule.atom_coords()

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.88972612],
       [ 0.        ,  0.        , -1.88972612]])

In [46]:
mf = scf.RHF(molecule)

In [48]:
%%time

mol_eq = optimize(mf)

geometric-optimize called with the following command line:
/Users/peter/Tatra/UHP/Code/Learning/venv11/lib/python3.11/site-packages/ipykernel_launcher.py --f=/Users/peter/Library/Jupyter/runtime/kernel-v2-17315T3PCGngoo48H.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **         


Geometry optimization cycle 1


Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   C   0.000000   0.000000   1.000000    0.000000  0.000000  0.000000
   C   0.000000   0.000000  -1.000000    0.000000  0.000000  0.000000
converged SCF energy = -149.98193184835
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O     0.0000000000    -0.0000000000     0.0000000000
1 C    -0.0000000000     0.0000000000    -0.7536217093
2 C    -0.0000000000     0.0000000000     0.7536217093
----------------------------------------------
cycle 1: E = -149.981931848  dE = -149.982  norm(grad) = 1.06578


Step    0 : Gradient = 6.153e-01/7.536e-01 (rms/max) Energy = -149.9819318483
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 2.50000e-01 3.32740e+00 3.32740e+00



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.000000   0.000000  -0.000000   -0.000000  0.000000 -0.000000
   C   0.000000   0.000000   1.119853    0.000000  0.000000  0.119853
   C  -0.000000  -0.000000  -1.119853   -0.000000 -0.000000 -0.119853
converged SCF energy = -150.193930756591
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O     0.0000000000    -0.0000000000     0.0000000000
1 C    -0.0000000000    -0.0000000000    -0.2458970364
2 C     0.0000000000     0.0000000000     0.2458970364
----------------------------------------------
cycle 2: E = -150.193930757  dE = -0.211999  norm(grad) = 0.347751


...


Geometry optimization cycle 6
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000 -0.000000 -0.000000
   C  -0.000000   0.000000   1.250063    0.000000  0.000000  0.005294
   C   0.000000  -0.000000  -1.250063   -0.000000 -0.000000 -0.005294
converged SCF energy = -150.245825547671
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O     0.0000000000     0.0000000000     0.0000000000
1 C    -0.0000000000    -0.0000000000    -0.0005891665
2 C    -0.0000000000    -0.0000000000     0.0005891665
----------------------------------------------
cycle 6: E = -150.245825548  dE = -7.43028e-05  norm(grad) = 0.000833207


Step    5 : Displace = 4.323e-03/5.294e-03 (rms/max) Trust = 3.000e-01 (+) Grad = 4.811e-04/5.892e-04 (rms/max) E (change) = -150.2458255477 (-7.430e-05) Quality = 1.080
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 2.50000e-01 6.28467e-01 3.32740e+00



Geometry optimization cycle 7
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.000000  -0.000000   0.000000   -0.000000 -0.000000 -0.000000
   C  -0.000000   0.000000   1.250559    0.000000  0.000000  0.000496
   C   0.000000  -0.000000  -1.250559    0.000000  0.000000 -0.000496
converged SCF energy = -150.245826110651
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0000000000    -0.0000000000     0.0000000000
1 C     0.0000000000     0.0000000000    -0.0000110039
2 C     0.0000000000     0.0000000000     0.0000110039
----------------------------------------------
cycle 7: E = -150.245826111  dE = -5.6298e-07  norm(grad) = 1.55618e-05


Step    6 : Displace = 4.051e-04/4.961e-04 (rms/max) Trust = 3.000e-01 (=) Grad = 8.985e-06/1.100e-05 (rms/max) E (change) = -150.2458261107 (-5.630e-07) Quality = 1.019
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 2.50000e-01 6.28467e-01 3.32740e+00
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 6.769 seconds


CPU times: user 4.09 s, sys: 197 ms, total: 4.29 s
Wall time: 6.84 s


In [49]:
mol_eq.atom_coords()

array([[-8.16725392e-14, -4.92852644e-14,  8.23367392e-13],
       [-5.70989290e-14,  1.98563920e-13,  2.36321441e+00],
       [ 1.30579764e-13, -1.26623951e-13, -2.36321441e+00]])